## Mouse 🐁 : M044

Session: M044_2024_12_04_09_30

## Tuning parameters for optimal RNN training

In [1]:
# imports
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import mat73
from IPython.display import display, Markdown

import pyaldata as pyal
import pylab
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from tools.curbd import curbd
import pylab
import importlib

from tools.dsp.preprocessing import preprocess
from tools.rnn_and_curbd.RNN_functions import *

np.random.seed(44)

#### Importing file

In [2]:
data_dir = "/data/bnd-data/raw/M044/M044_2024_12_04_09_30"
mat_file = "M044_2024_12_04_09_30_pyaldata.mat"
fname = os.path.join(data_dir, mat_file)

df = pyal.mat2dataframe(fname, shift_idx_fields=True)
mouse = mat_file.split('_')[0]

field values_before_camera_trigger could not be converted to int.
field idx_before_camera_trigger could not be converted to int.
array field all_KSLabel could not be converted to int.


#### Preprocessing

In [3]:
df_ = preprocess(df, only_trials=True)
df_ = pyal.select_trials(df_, "idx_trial_end > 30365")  # Remove first 5 minutes because the switch was off

# if you dont want to run it on all trials
# df_ = pyal.select_trials(df_, f"values_Sol_direction < 4") 

/home/zms24/.conda/envs/zofia_earthquake_env/earthquake-analysis/PyalData/pyaldata/firing_rates.py:108: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn(


Combined every 3 bins
Resulting all_spikes ephys data shape is (NxT): (474, 133)


#### Seperate regions

In [4]:
# Separate columns of 'all_spikes' to M1 and Dls (dorsolateral striatum)
brain_areas = ["Dls_rates", "M1_rates"]
df_["M1_rates"] = [df_["all_rates"][i][:,300:] for i in range(len(df_))]
df_["Dls_rates"] = [df_["all_rates"][i][:,0:300] for i in range(len(df_))]

#### Variables

In [5]:
# perturbation time
perturbation_time = df_.idx_sol_on[0]
perturbation_time_seconds = perturbation_time * df_.bin_size[0]

# solenoid angles
sol_angles: list = df_.values_Sol_direction.unique()
sol_angles.sort()

# number of trials
num_trials = len(df_)
print(f"Number of trials: {num_trials}")

Number of trials: 493


#### Concat trials in time

In [6]:
concat_trials = pyal.concat_trials(df_, signal = "all_rates")

## RNN training loop

In [ ]:
dtFactors = [1, 2, 3, 4, 5, 6]
tauRNNs = [0.01, 0.05, 0.1, 0.2, 0.3]
ampInWNs = [0.0001, 0.001]

rnn_models = []

for dtFactor in dtFactors:
    reset_points, trial_len = get_reset_points(df_, concat_trials, brain_areas, dtFactor)
    activity = np.transpose(concat_trials)
    regions = get_regions(df_, brain_areas)

    for tauRNN in tauRNNs:
        for ampInWN in ampInWNs:
                rnn_model = RNN(activity, reset_points, regions, df_, mouse, graph=False,
                dtFactor = dtFactor, ampInWN= ampInWN, tauRNN = tauRNN, nRunTrain=20) # 20 should be suffucient to get a good estimate of the parameters
                params = [dtFactor, tauRNN, ampInWN, trial_len, reset_points]
                result = [params, rnn_model]
                rnn_models.append(result)


Trial length: 133


#### Organizing RNN training results